In [ ]:
from bs4 import BeautifulSoup
import urllib
from functools import partial
import mysql.connector
import re

### Funções para coleta de URLs, títulos, e textos das páginas

In [ ]:
def getPageSoup(url):
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,} 
    request=urllib.request.Request(url,None,headers) 
    response = urllib.request.urlopen(request)
    soup = BeautifulSoup(response, 'html.parser')
    return soup

def getPageText(soup):
    page_text = soup.find_all('p')
    page_string = ""
    for p in page_text:
        page_string += p.get_text()
    return page_string

def getPageTitle(soup):
    return soup.title.string

def getAllLinks(soup):
    links =[]
    all_links = soup.find_all('a')
    for link in all_links:
        links.append(link.get('href'))
    return links
    

### Declaração da URL e variáveis iniciais

In [ ]:
original_url = "https://johnpistelli.com/"
soup = getPageSoup(original_url)
page_links = getAllLinks(soup)
page_links_set = set()
page_links_set.add(original_url)

### Loop inicial para preencher o Set com todos os links encontrados pelo crawling

In [ ]:
max_visited_pages = 50000 # número de links máximo que serão percorridos
counter = 0
for page in page_links:
    if counter > max_visited_pages:
        break
    try:
        soup = getPageSoup(page)
        links = getAllLinks(soup)
        for link in links:
            page_links_set.add(link)
            counter += 1
    except:
        print("bad request")
        

print("Set completed")    

### Conexão com o DataBase (mysql)

In [ ]:
import sqlite3
con = sqlite3.connect('SQL.db')
cur = con.cursor()

### Criação da tabela (rodar apenas uma vez)

In [ ]:
cur.execute("DROP TABLE websites")
cur.execute("DROP TABLE words")
cur.execute("DROP TABLE website_has_word")

In [ ]:
cur.execute("CREATE TABLE websites(website_id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT, url VARCHAR(150),website_content TEXT,website_title TEXT)")

In [ ]:
con.commit()

### Loop para o preenchimento do Database com o título, URL e texto de cada página

In [ ]:
for link in page_links_set:
    try:
        soup = getPageSoup(link)
        page_text = getPageText(soup)
        page_title = getPageTitle(soup)
        cur.execute(f"INSERT INTO websites(url, website_content, website_title) VALUES('{link}', '{page_text}', '{page_title}')")
        con.commit()
    except:
        print("An exception occurred")
connection.close()